<a href="https://colab.research.google.com/github/dlkt101101/AMATH445/blob/main/AMATH445_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMATH 445
## Assignment 2
Prepared by: Darren Alexander Lam Kin Teng (20977843)

# Question 1

In [29]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import math
import torch
import torch.nn as nn
from torch.nn import ReLU, Tanh
import torch.optim as optim
from sklearn.model_selection import KFold
from itertools import product

In [2]:
with open('/content/A2Q1_data_regression.pkl', 'rb') as file:
  data_regression = pickle.load(file)

with open('/content/A2Q1_data.pkl', 'rb') as file:
  data_classification = pickle.load(file)

In [3]:
print(data_regression.keys(), data_classification.keys())

dict_keys(['X_train', 'X_test', 'y_train', 'y_test']) dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


In [4]:
# regression
X_train_regression = data_regression['X_train']
y_train_regression = data_regression['y_train']
X_test_regression = data_regression['X_test']
y_test_regression = data_regression['y_test']

# classification
X_train_classification = data_classification['X_train']
y_train_classification = data_classification['y_train']
X_test_classification = data_classification['X_test']
y_test_classification = data_classification['y_test']

In [7]:
print('Checking Regression dataset split:')
print(X_train_regression.shape[0]/(X_train_regression.shape[0]+X_test_regression.shape[0]))

print('Checking Classification dataset split:')
print(X_train_classification.shape[0]/(X_train_classification.shape[0]+X_test_classification.shape[0]))

Checking Regression dataset split:
0.847457627118644
Checking Classification dataset split:
0.847457627118644


## Model Development

There will only be one node for both cases of binary classification and regression.

The input layer will contain the same number of nodes as our explanatory variables.

The number of neurons per hidden layer is based on a rule of thumb:
$Neurons = \frac{2}{3}(\text{neurons in input layer})+1$
(https://www.heatonresearch.com/2017/06/01/hidden-layers.html)

In [56]:
X_train_classification = X_train_classification.astype(float)
X_train_regression = X_train_regression.astype(float)

In [90]:
input_layer_regression = X_train_regression.shape[1]
input_layer_classification = X_train_classification.shape[1]
# N_neurons_regression = math.floor(2/3 * input_layer_regression) + 1
# N_neurons_classification = math.floor(2/3 * input_layer_classification) + 1
N_neurons_regression = 10
N_neurons_classification = 10
output_layer = 1

EPOCHS = 100

In [82]:
class NeuralNetwork(nn.Module):
    def __init__(self, problem="classification", activation_function='relu'):
        super().__init__()
        self.problem = problem
        self.activation_function_type = activation_function

        if problem == "classification":
          INPUT_SIZE, N_NEURONS = input_layer_classification, N_neurons_classification
        elif problem == "regression":
          INPUT_SIZE, N_NEURONS = input_layer_regression, N_neurons_regression
        else:
          print("Invalid problem. Problem must be 'regression' or 'classification'.")

        # building the neural network
        self.layers = nn.Sequential(
            nn.Linear(INPUT_SIZE, N_NEURONS),
            Tanh() if self.activation_function_type in ['mixed', 'tanh'] else ReLU(),
            nn.Linear(N_NEURONS, N_NEURONS),
            ReLU() if self.activation_function_type in ['mixed', 'relu'] else Tanh(),
            nn.Linear(N_NEURONS, N_NEURONS),
            ReLU() if self.activation_function_type in ['mixed', 'tanh'] else Tanh(),
            nn.Linear(N_NEURONS, output_layer))

    def forward(self, x):
      logits = self.layers(x)
      if self.problem == "classification":
        return torch.sigmoid(logits)
      return logits

In [27]:
kf = KFold(n_splits = 5, random_state=123, shuffle=True)

In [85]:
def train_evaluate_model_cv(problem_type='classification', learning_rate = [0.01, 0.001],
                            momentum = [0.0, 0.9], activation_function = ['relu', 'tanh', 'mixed']):

  X_raw = X_train_classification if problem_type == 'classification' else X_train_regression
  y_raw = y_train_classification if problem_type == 'classification' else y_train_regression

  X = torch.tensor(X_raw.values, dtype=torch.float32)
  y = torch.tensor(y_raw.values, dtype=torch.float32)

  parameter_grid = {'lr' : learning_rate,
                    'mom' : momentum,
                    'act' : activation_function}

  combinations = list(product(parameter_grid['lr'], parameter_grid['mom'], parameter_grid['act']))

  # store the best hyper parameters
  best_loss = math.inf
  best_parameters = {}

  for lr, mom, act in combinations:
    losses = []
    for i, (train_index, val_index) in enumerate(kf.split(X)):
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      model = NeuralNetwork(problem=problem_type, activation_function=act)
      optimizer = optim.Adam(model.parameters(), lr=lr, betas=(mom, 0.999))

      # specify the criterion for classification and regression
      if problem_type == 'regression':
        criterion = nn.MSELoss()
      else:
        criterion = nn.BCELoss()

      model.train()
      for epoch in range(EPOCHS):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.view_as(outputs))
        loss.backward()
        optimizer.step()

      # model evaluation
      model.eval()
      with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.view_as(val_outputs))
        losses.append(val_loss.item())

    avg_fold_loss = np.mean(losses)

    if avg_fold_loss < best_loss:
      best_loss = avg_fold_loss
      best_parameters = {'lr' : lr,
                         'mom' : mom,
                         'act' : act}

  return (best_loss, best_parameters)


In [86]:
class_params = train_evaluate_model_cv(problem_type='classification')

In [87]:
regress_params = train_evaluate_model_cv(problem_type='regression')

Below we get the results from our deep learning models for both regression and classification along with their best parameters.

In [88]:
class_params

(np.float64(0.6278908967971801), {'lr': 0.001, 'mom': 0.9, 'act': 'relu'})

In [89]:
regress_params

(np.float64(270.81939086914065), {'lr': 0.01, 'mom': 0.0, 'act': 'mixed'})

In [74]:
y_train_regression

,Viability_at_time_of_observation_(%)
86,68.0
290,93.0
40,58.0
6,85.0
447,59.0
...,...
492,85.0
283,85.5
7,76.0
337,92.1
